In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance as dist
import statistics
from math import degrees, radians, cos, sin, asin, sqrt,pi,atan, asin, atan2

In [2]:
radar = pd.read_csv('radarwithacceleration.csv')
video = pd.read_csv('videowithacceleration.csv')
print(radar)

       Unnamed: 0        lat        lon   v_n   v_e           a_n       a_e  \
0               0  60.160418  24.921638 -8.75  8.55  0.000000e+00  0.000000   
1               0  60.160562  24.922052  8.34  0.45  0.000000e+00  0.000000   
2               0  60.160589  24.921923 -7.95 -0.75  0.000000e+00  0.000000   
3               0  60.160640  24.922041  7.94  1.25  0.000000e+00  0.000000   
4               0  60.160719  24.922022  7.94  1.65  0.000000e+00  0.000000   
...           ...        ...        ...   ...   ...           ...       ...   
52924           0  60.160740  24.921974 -2.05 -0.45 -7.340653e-12  0.000000   
52925           0  60.160789  24.921927 -9.05 -2.85 -7.806519e-11  0.000000   
52926           0  60.161027  24.921871 -8.95 -2.15  3.715372e-11  0.000002   
52927           0  60.161394  24.921958  7.55  2.05 -1.256748e-11  0.000000   
52928           0  60.161393  24.922006  7.55  2.05 -1.313169e-11  0.000000   

        len     id  lane  class  cyc_ago  quality  

In [3]:
from haversine import haversine_vector, Unit

In [5]:
i = 0 # i kertoo mitä havaintoriviä käydään läpi videodatasta
j = 0 # j kertoo mitä havaintoriviä käydään läpi radardatasta 
k = 0 
frvideo = np.unique(video.frame.values)
frradar = np.unique(radar.tstamp.values)
fr_lenvideo = frvideo.shape[0]
fr_lenradar = frradar.shape[0]
videotstamp = [0]*fr_lenvideo# alustetaan videoaikaleimojen määrä
videotstamp[0] = (radar.tstamp[0]-9500) # tämä videostamps[0] on visuaalisesti itse määritelty
fused_tracking = pd.DataFrame({'lat': [],'lon': [], 'type': [],'tstamp': [], 'quality': [],'radarID': [], 'videoID':[]})
while j < (fr_lenradar) and i < (fr_lenvideo): # j täytyy olla pienempi kuin radar-havaintojen määärän ja i:n pienempi kuin video-havaintojen
    videodetects = video.loc[(video['frame'] == frvideo[i])&(video['lat'] < 60.161)].values
    radardetects = radar.loc[(radar['tstamp'] == frradar[j])].values
    time_difference = frradar[j]-videotstamp[i]
    if abs(time_difference)<500:
        velocity_video = videodetects[:,[5,6]]
        points_video = videodetects[:,[1,2]]
        videoIDs =  videodetects[:,[5]]
        vehicle = videodetects[:,[2]]
        radarquality = radardetects[:,[12]]
        velocity_radar = radardetects[:,[3,4]]
        points_radar = radardetects[:,[1,2]]
        points_video = points_video.tolist()
        points_radar = points_radar.tolist()
        acceleration_radar = radardetects[:,[5,6]]
        acceleration_video = videodetects[:,[7,8]]
        radarIDs = radardetects[:,[8]]+(k*265)
        velocitydistance = dist.cdist(velocity_video, velocity_radar)
        accelerationdistance = dist.cdist(acceleration_video, acceleration_radar)
        distance = haversine_vector(points_radar,points_video, Unit.METERS, comb=True)
        quality = 8*distance + velocitydistance #+ accelerationdistance
        rows = quality.min(axis=1).argsort()
        cols = quality.argmin(axis=1)[rows]
        usedRows = set()
        usedCols = set()
        #distances = []
        for (row, col) in zip(rows, cols):
            if row in usedRows or col in usedCols:
                continue
           # if distance[row][col] < 10 and velocitydistance[row][col] < 15:
            vehicle = pd.DataFrame({'lat': [radardetects[col][1]],'lon': [radardetects[col][2]], 'type': [videodetects[row][3]],'tstamp': [frradar[j]], 'quality': [radardetects[col][12]], 'radarID': [radarIDs[col][0]], 'videoID': [videodetects[row][9]]})
            fused_tracking = fused_tracking.append(vehicle)
            usedRows.add(row)
            usedCols.add(col)
                #distances.append(distance[row][col])
        unusedRows = set(range(0, quality.shape[0])).difference(usedRows)
        unusedCols = set(range(0, quality.shape[1])).difference(usedCols)
        #if len(distances)>10:
                #med_dist= statistics.median(distances)
                #videotstamp[i] = videotstamp[i] + 0.1*time_difference/(1+med_dist)
        for col in unusedCols:
            #if radardetects[col][12] > 10:
            vehicle = pd.DataFrame({'lat': [radardetects[col][1]],'lon': [radardetects[col][2]], 'type': ['unknown'],'tstamp': [frradar[j]], 'quality': [radardetects[col][12]], 'radarID': [radardetects[col][8]], 'videoID': [' ']})
            fused_tracking = fused_tracking.append(vehicle)
            usedCols.add(col) 
            unusedCols = set(range(0, quality.shape[1])).difference(usedCols)
    if time_difference < 0:
        j = j+1
        if j < fr_lenradar:
            time_difference = frradar[j]-videotstamp[i]
            print()
    while time_difference > 0 and i < fr_lenvideo:
        i = i + 1
        if i < fr_lenvideo:
            videotstamp[i] = videotstamp[i-1]+((frvideo[i]-frvideo[i-1])*(1000/30))
            time_difference = frradar[j]-videotstamp[i]
            moment = (videotstamp[i]-radar.tstamp.min())/(radar.tstamp.max()-radar.tstamp.min())*100
            print(moment,'%')
fused_tracking.to_csv('velocity_and_distance_fused.csv')
print(fused_tracking)    

-3.0504080182917743 %
-3.039667171303099 %
-3.0289263243144235 %
-3.018185477325748 %
-3.0074446303370728 %
-2.9967037833483974 %
-2.985962936359722 %
-2.9752220893710466 %
-2.964481242382371 %
-2.9537403953936963 %
-2.942999548405021 %
-2.932258701416345 %
-2.9215178544276696 %
-2.9107770074389943 %
-2.9000361604503193 %
-2.889295313461644 %
-2.8785544664729685 %
-2.867813619484293 %
-2.8570727724956178 %
-2.8463319255069424 %
-2.835591078518267 %
-2.8248502315295916 %
-2.814109384540916 %
-2.8033685375522412 %
-2.792627690563566 %
-2.78188684357489 %
-2.7711459965862146 %
-2.7604051495975392 %
-2.7496643026088643 %
-2.738923455620189 %
-2.7281826086315135 %
-2.717441761642838 %
-2.7067009146541627 %
-2.6959600676654873 %
-2.685219220676812 %
-2.6744783736881366 %
-2.663737526699461 %
-2.652996679710786 %
-2.642255832722111 %
-2.6315149857334355 %
-2.6207741387447596 %
-2.6100332917560842 %
-2.599292444767409 %
-2.588551597778734 %
-2.5778107507900585 %
-2.567069903801383 %
-2.5563290

1.1600040012689672 %
1.1707448482576424 %

1.1814856952463177 %
1.1922265422349931 %

1.2029673892236685 %
1.2137082362123437 %
1.224449083201019 %

1.2351899301896947 %

1.2459307771783699 %
1.2566716241670453 %
1.2674124711557206 %

1.278153318144396 %
1.2888941651330714 %

1.2996350121217466 %
1.3103758591104222 %

1.3211167060990974 %
1.3318575530877728 %
1.3425984000764482 %

1.3533392470651235 %
1.364080094053799 %
1.374820941042474 %
1.3855617880311497 %

1.396302635019825 %
1.4070434820085003 %

1.4177843289971757 %
1.428525175985851 %
1.4392660229745264 %

1.4500068699632016 %
1.4607477169518772 %
1.4714885639405526 %

1.4822294109292278 %
1.4929702579179032 %

1.5037111049065786 %

1.514451951895254 %
1.5251927988839293 %
1.5359336458726047 %
1.5466744928612801 %

1.5574153398499553 %
1.5681561868386307 %
1.5788970338273063 %

1.5896378808159817 %
1.6003787278046566 %
1.611119574793332 %

1.6218604217820076 %
1.632601268770683 %

1.6433421157593584 %
1.6540829627480336 %

1.6

5.434861102761761 %
5.4456019497504355 %

5.456342796739111 %
5.467083643727786 %
5.477824490716462 %
5.488565337705138 %

5.499306184693813 %

5.510047031682488 %
5.520787878671164 %
5.531528725659839 %

5.5422695726485145 %
5.553010419637189 %

5.563751266625864 %

5.57449211361454 %
5.585232960603215 %
5.595973807591891 %

5.606714654580566 %
5.617455501569242 %

5.628196348557917 %
5.638937195546593 %
5.6496780425352675 %

5.660418889523943 %
5.671159736512618 %

5.681900583501294 %
5.692641430489969 %
5.703382277478644 %

5.71412312446732 %
5.724863971455995 %
5.735604818444671 %

5.746345665433346 %
5.7570865124220205 %
5.767827359410696 %

5.778568206399372 %
5.789309053388047 %
5.800049900376723 %
5.810790747365398 %
5.821531594354074 %

5.832272441342749 %

5.8430132883314245 %
5.8537541353200995 %

5.864494982308774 %

5.87523582929745 %
5.885976676286125 %

5.896717523274801 %
5.907458370263476 %
5.918199217252152 %

5.928940064240827 %
5.939680911229503 %

5.950421758218178


9.978240008319478 %
9.988980855308153 %
9.999721702296828 %

10.010462549285503 %
10.02120339627418 %

10.031944243262854 %
10.04268509025153 %

10.053425937240206 %
10.06416678422888 %
10.074907631217556 %
10.085648478206231 %
10.096389325194908 %
10.107130172183581 %

10.117871019172258 %

10.128611866160933 %
10.139352713149608 %

10.150093560138282 %
10.16083440712696 %
10.171575254115634 %

10.182316101104309 %
10.193056948092986 %

10.20379779508166 %
10.214538642070336 %

10.22527948905901 %
10.236020336047687 %
10.24676118303636 %

10.257502030025037 %
10.268242877013712 %

10.278983724002387 %
10.289724570991062 %

10.300465417979739 %
10.311206264968414 %

10.321947111957089 %
10.332687958945765 %

10.34342880593444 %
10.354169652923115 %

10.36491049991179 %
10.375651346900467 %
10.38639219388914 %
10.397133040877817 %
10.407873887866492 %
10.418614734855169 %
10.429355581843842 %

10.440096428832518 %

10.450837275821193 %
10.461578122809868 %

10.472318969798543 %
10.4830


14.177911180891545 %
14.188652027880218 %

14.199392874868893 %
14.21013372185757 %
14.220874568846245 %

14.23161541583492 %

14.242356262823597 %
14.253097109812272 %
14.263837956800945 %
14.274578803789623 %
14.285319650778296 %

14.296060497766971 %
14.306801344755648 %

14.317542191744323 %
14.328283038732998 %

14.339023885721675 %
14.34976473271035 %

14.360505579699023 %
14.371246426687701 %

14.381987273676375 %
14.392728120665051 %
14.403468967653726 %

14.414209814642401 %
14.424950661631078 %

14.435691508619753 %
14.446432355608426 %

14.457173202597104 %
14.467914049585778 %

14.478654896574453 %
14.48939574356313 %
14.500136590551804 %

14.51087743754048 %
14.521618284529156 %

14.53235913151783 %
14.543099978506504 %

14.553840825495183 %
14.564581672483856 %

14.57532251947253 %
14.586063366461207 %
14.596804213449882 %

14.607545060438559 %
14.618285907427234 %

14.629026754415907 %
14.639767601404586 %

14.650508448393259 %
14.661249295381934 %

14.67199014237061 %


...


81.10413451012857 %
81.11487535711726 %

81.12561620410594 %
81.13635705109459 %
81.14709789808327 %

81.15783874507196 %
81.16857959206062 %

81.1793204390493 %
81.19006128603799 %

81.20080213302666 %
81.21154298001532 %

81.222283827004 %
81.23302467399269 %

81.24376552098136 %
81.25450636797004 %
81.2652472149587 %
81.27598806194737 %

81.28672890893606 %

81.29746975592474 %
81.3082106029134 %

81.31895144990209 %
81.32969229689076 %

81.34043314387944 %
81.3511739908681 %

81.36191483785679 %
81.37265568484547 %
81.38339653183414 %

81.3941373788228 %
81.40487822581149 %

81.41561907280015 %
81.42635991978884 %
81.43710076677752 %
81.44784161376619 %

81.45858246075485 %

81.46932330774354 %
81.48006415473222 %
81.49080500172089 %

81.50154584870957 %
81.51228669569825 %

81.5230275426869 %

81.53376838967559 %
81.54450923666427 %
81.55525008365294 %
81.56599093064162 %

81.5767317776303 %
81.58747262461897 %
81.59821347160764 %
81.60895431859632 %

81.619695165585 %
81.6304360


85.4541775405421 %
85.46491838753077 %

85.47565923451945 %
85.48640008150812 %
85.49714092849679 %

85.50788177548547 %
85.51862262247415 %

85.52936346946282 %
85.5401043164515 %

85.55084516344017 %
85.56158601042884 %

85.57232685741752 %
85.5830677044062 %
85.59380855139487 %
85.60454939838355 %

85.61529024537224 %

85.6260310923609 %
85.63677193934957 %
85.64751278633825 %

85.65825363332694 %

85.6689944803156 %
85.67973532730429 %
85.69047617429295 %

85.70121702128162 %
85.7119578682703 %

85.72269871525899 %
85.73343956224765 %

85.74418040923634 %
85.754921256225 %

85.76566210321369 %
85.77640295020235 %
85.78714379719104 %

85.79788464417972 %
85.80862549116839 %

85.81936633815705 %
85.83010718514574 %

85.8408480321344 %
85.85158887912309 %

85.86232972611177 %
85.87307057310043 %
85.8838114200891 %

85.89455226707778 %
85.90529311406647 %

85.91603396105513 %
85.92677480804382 %
85.9375156550325 %

85.94825650202115 %

85.95899734900983 %
85.96973819599852 %
85.980479


89.81496141794429 %
89.82570226493297 %
89.83644311192165 %
89.84718395891032 %

89.85792480589899 %
89.86866565288767 %

89.87940649987635 %
89.89014734686502 %

89.9008881938537 %
89.91162904084237 %
89.92236988783104 %

89.93311073481972 %
89.9438515818084 %

89.95459242879707 %
89.96533327578575 %

89.97607412277442 %
89.9868149697631 %
89.99755581675177 %

90.00829666374045 %
90.01903751072913 %

90.0297783577178 %
90.04051920470647 %

90.05126005169515 %
90.06200089868382 %

90.0727417456725 %
90.08348259266118 %
90.09422343964985 %

90.10496428663853 %
90.1157051336272 %

90.12644598061587 %
90.13718682760455 %
90.14792767459323 %
90.1586685215819 %
90.16940936857058 %

90.18015021555925 %

90.19089106254793 %
90.2016319095366 %
90.21237275652528 %

90.22311360351397 %
90.23385445050263 %
90.2445952974913 %

90.25533614447998 %
90.26607699146865 %

90.27681783845733 %
90.28755868544602 %
90.29829953243468 %

90.30904037942335 %
90.31978122641203 %
90.33052207340072 %

90.341262

94.33685839351914 %
94.34759924050782 %
94.3583400874965 %

94.36908093448517 %

94.37982178147385 %
94.39056262846253 %

94.40130347545119 %
94.41204432243987 %

94.42278516942855 %
94.43352601641722 %

94.4442668634059 %
94.45500771039458 %
94.46574855738325 %

94.47648940437192 %
94.4872302513606 %
94.49797109834928 %
94.50871194533795 %

94.51945279232663 %
94.5301936393153 %

94.54093448630397 %
94.55167533329265 %
94.56241618028133 %

94.57315702727 %
94.58389787425868 %

94.59463872124735 %
94.60537956823603 %

94.6161204152247 %
94.62686126221338 %
94.63760210920206 %

94.64834295619073 %
94.65908380317941 %

94.66982465016808 %
94.68056549715675 %

94.69130634414543 %
94.70204719113411 %

94.71278803812278 %
94.72352888511146 %
94.73426973210013 %

94.74501057908881 %
94.75575142607748 %

94.76649227306616 %
94.77723312005483 %

94.78797396704351 %
94.79871481403218 %

94.80945566102086 %
94.82019650800953 %
94.83093735499821 %

94.8416782019869 %
94.85241904897556 %

94.86315


98.75134666320173 %
98.7620875101904 %
98.77282835717908 %

98.78356920416775 %
98.79431005115643 %

98.80505089814511 %
98.81579174513378 %

98.82653259212245 %
98.83727343911113 %
98.84801428609981 %
98.85875513308848 %
98.86949598007716 %

98.88023682706584 %
98.8909776740545 %

98.90171852104318 %
98.91245936803186 %

98.92320021502053 %
98.93394106200921 %

98.9446819089979 %
98.95542275598656 %
98.96616360297523 %

98.97690444996391 %
98.98764529695259 %

98.99838614394126 %
99.00912699092994 %
99.01986783791861 %

99.03060868490728 %
99.04134953189596 %

99.05209037888464 %
99.06283122587331 %

99.07357207286199 %
99.08431291985066 %

99.09505376683934 %
99.10579461382801 %

99.11653546081669 %
99.12727630780537 %

99.13801715479404 %
99.14875800178272 %
99.15949884877139 %
99.17023969576006 %
99.18098054274874 %

99.19172138973742 %
99.20246223672609 %

99.21320308371477 %
99.22394393070344 %

99.23468477769212 %
99.24542562468079 %
99.25616647166947 %

99.26690731865816 %
99.